In [1]:
# Executing these “magic commands” will inform your notebook to actively
# reload all imported modules and packages as they are modified.
%load_ext autoreload

In [2]:
%reload_ext autoreload

In [3]:
# import necessary modules
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import glob
import os
import sys

In [4]:
import Compare_IPA as ipa

In [5]:
desc_transcript_files = glob.glob('transcriptions/descriptive/*.txt') # take in all desc filepaths
presc_transcript_file = glob.glob('transcriptions/prescriptive/*.txt') # take in presc transc file

In [6]:
test_file = desc_transcript_files[0]

In [7]:
test_file

'transcriptions/descriptive/partic13.txt'

In [8]:
test_class = ipa.Partic(test_file)

In [10]:
test_class.dictionary

{'full_path': 'transcriptions/descriptive/partic13.txt',
 'file_name': 'partic13',
 'prefix': '',
 'extention': '.txt',
 'raw_transcript': "i-se # kom-bi-na-si̯on # i-ni-si̯o # e-ðu-ka-si̯on # as-ta # o-ho # ka-si # 'u-saɾ # fun-si̯o-naɾ # kan-taɾ # ai̯-ju-ða # om-bɾe # es-ta # sa-βe # o-poɾ-tu-ni-ðað # to-ke # el # su # eɽ-ma-na # es-ta # ta-sa # im-poɾ-tan-te # ke # el # tan-to",
 'clean_transcript': ['i-se',
  'kom-bi-na-si̯on',
  'i-ni-si̯o',
  'e-ðu-ka-si̯on',
  'as-ta',
  'o-ho',
  'ka-si',
  'u-saɾ',
  'fun-si̯o-naɾ',
  'kan-taɾ',
  'ai̯-ju-ða',
  'om-bɾe',
  'es-ta',
  'sa-βe',
  'o-poɾ-tu-ni-ðað',
  'to-ke',
  'el',
  'su',
  'eɽ-ma-na',
  'es-ta',
  'ta-sa',
  'im-poɾ-tan-te',
  'ke',
  'el',
  'tan-to']}

In [ ]:
partic_IDs = [(x + 1) for x in range(15)]

In [ ]:
partic_IDs